In [1]:
#criei o repositório para guardar as anotações do estudos de modelos (principalmente de regressão

#Regressão Polinomial
#criação de dados quadráticos não lineares com ruído

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

m = 100 # instâncias de treinamento
X = 6 * np.random.rand(m,1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m,1)

In [2]:
#para ajustar aos dados, usamos a classe PolynomialFeatures do SKLearn para transformar dados de treino, adicionando o quadrado de cada característica no conjunto de treinamento como uma nova característica

from sklearn.preprocessing import PolynomialFeatures
# include_bias=False: Evita adicionar uma coluna extra de 1s (que representa o viés/x^0).
# isso é feito porque a maioria dos modelos do SKLearn (como LinearRegression) já
# calcula o intercepto automaticamente, evitando assim duplicidade no termo de viés.
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)
X[0]

array([-1.54051377])

In [3]:
X_poly[0]

array([-1.54051377,  2.37318268])

In [ ]:
# X_poly tem a caracterísitica original de X mais o quadrado dessa característica, dando para ajustar os dados no LinearRegression

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)

lin_reg.intercept_, lin_reg.coef_

In [ ]:
# em uma regressão polinomial, o modelo consegue encontrar relações entre várias caracterísiticas, coisa que a regressão linear não consegue. Isso por conta do PolynomialFeatures(degree=d)

#PolynomialFeatures(degree=3) não adiciona somente as caracteristicas a^2, a^3, b^2, b^3, mas também as combinações de ab, a^2b, ab^2

plt.scatter(X , y)
plt.plot(X , lin_reg.predict(X), color='red')

In [ ]:
#antes de plotar as curvas de aprendizado, o livro fala sobre utilizar a validação cruzada para verificar se o modelo está se sobre ou subajustando
#primeiro preciso dividir os dados em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

from sklearn.model_selection import cross_val_score
scores_test = cross_val_score(lin_reg, X_train, y_train, cv=10, scoring='neg_mean_squared_error')